In [1]:
from pathlib import Path

import numpy as np
from astropy.io import ascii
import matplotlib.pyplot as plt

from dask.distributed import Client
import dask.array
from dask.dataframe.utils import make_meta

from hats import read_hats
from hats.inspection import plot_pixels
from hats_import.catalog.file_readers import CsvReader
from hats_import.margin_cache.margin_cache_arguments import MarginCacheArguments
from hats_import.pipeline import ImportArguments, pipeline_with_client

import lsdb

from catalog_filtering import bandFilterLenient, contains_PM

print("Imported Libraries")

Imported Libraries


In [2]:
DES_HATS_DIR = Path("../../../shared/hats/catalogs/des/des_dr2")
GAIA_HATS_DIR = Path("../../../shared/hats/catalogs/gaia_dr3/gaia")

CATALOG_DIR = Path("../../catalogs")

GAIA_PM_NAME = "gaia_dr3_pm_greater_100"
GAIA_PM_DIR = CATALOG_DIR / GAIA_PM_NAME
GAIA_MARGIN_CACHE_NAME = "gaia_margin_cache_18_arcsec"
GAIA_MARGIN_CACHE_DIR = CATALOG_DIR / GAIA_MARGIN_CACHE_NAME

DES_LIGHT_NAME = "des_light"
DES_LIGHT_DIR = CATALOG_DIR / DES_LIGHT_NAME

XMATCH_NAME = "des_dr2_x_gaia_dr3"
XMATCH_DIR = CATALOG_DIR / XMATCH_NAME

print("Defined directories")

Defined directories


In [3]:
bandList = ['G','R','I','Z','Y']
class_star = None
spread_model = 0.05
magnitude_error = 0.05
check_flags = True
check_invalid_mags = True
query_string = bandFilterLenient(bandList,classStar=class_star,spreadModel=spread_model,magError=magnitude_error,flag=check_flags,invalidMags=check_invalid_mags)
des_cols = (
    [f'CLASS_STAR_{band}' for band in bandList] + 
    [f'FLAGS_{band}' for band in bandList] + 
    ['RA','DEC','COADD_OBJECT_ID'] + 
    [f'SPREAD_MODEL_{band}' for band in bandList] + 
    [f'WAVG_MAG_PSF_{band}' for band in bandList] + 
    [f'WAVG_MAGERR_PSF_{band}' for band in bandList]
)
pm_speed_min = 2000 #units are milliseconds per year
pm_speed_max = 10**5
max_neighbor_dist = 18
print("Defined filter variables")

Defined filter variables


In [4]:
des_dr2_light = lsdb.read_hats(DES_LIGHT_DIR)
gaia_dr3_pm_greater_100 = lsdb.read_hats(GAIA_PM_DIR, margin_cache=GAIA_MARGIN_CACHE_DIR)
print("Obtained catalogs")

Obtained catalogs


In [5]:
xmatch = des_dr2_light.crossmatch(gaia_dr3_pm_greater_100,
    suffixes=("_des","_gaia"),
    n_neighbors=100, radius_arcsec=18
)
xmatch

,CLASS_STAR_G_des,CLASS_STAR_R_des,CLASS_STAR_I_des,CLASS_STAR_Z_des,CLASS_STAR_Y_des,FLAGS_G_des,FLAGS_R_des,FLAGS_I_des,FLAGS_Z_des,FLAGS_Y_des,RA_des,DEC_des,COADD_OBJECT_ID_des,SPREAD_MODEL_G_des,SPREAD_MODEL_R_des,SPREAD_MODEL_I_des,SPREAD_MODEL_Z_des,SPREAD_MODEL_Y_des,WAVG_MAG_PSF_G_des,WAVG_MAG_PSF_R_des,WAVG_MAG_PSF_I_des,WAVG_MAG_PSF_Z_des,WAVG_MAG_PSF_Y_des,WAVG_MAGERR_PSF_G_des,WAVG_MAGERR_PSF_R_des,WAVG_MAGERR_PSF_I_des,WAVG_MAGERR_PSF_Z_des,WAVG_MAGERR_PSF_Y_des,source_id_gaia,ra_gaia,dec_gaia,pmra_gaia,pmdec_gaia,_dist_arcsec
npartitions=1587,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Order: 4, Pixel: 0",double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int16[pyarrow],int16[pyarrow],int16[pyarrow],int16[pyarrow],int16[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow]
"Order: 5, Pixel: 8",...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Order: 3, Pixel: 752",...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Order: 3, Pixel: 767",...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [ ]:
# This is the code I ran to save the crossmatch, though 

with Client(n_workers=3):
    xmatch.to_hats(catalog_name = XMATCH_NAME,
        base_catalog_path = XMATCH_DIR
    )
print("Saved crossmatch to catalogs folder.")